# Machine Learning for Demand Forecasting

### Use case - predicting the demand for items at a libary

In [1]:
sfOptions = {
  "sfURL" : "datalytyx.east-us-2.azure.snowflakecomputing.com",
  "sfAccount" : "datalytyx",
  "sfUser" : "WILLHOLTAM",
  "sfPassword" : "04MucSfLV",
  "sfRole": "DATABRICKS",
  "sfDatabase" : "DATABRICKS_DEMO",
  "sfSchema" : "SEATTLE_LIBRARY",
  "sfWarehouse" : "DATASCIENCE_WH"
}
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

In [2]:
# spark.conf.set("spark.executor.memory", "10g")
# spark.conf.set("spark.databricks.io.cache.enabled", "true")
# #spark.conf.set("spark.databricks.io.cache.maxDiskUsage", "10g")
# spark.conf.set("spark.databricks.io.cache.maxMetaDataCache", "10g")
# # =true
# spark.databricks.io.cache.maxDiskUsage 10g
# spark.databricks.io.cache.maxMetaDataCache 10g

# spark.conf.set(
#   "fs.azure.account.key.dlxhackathon1.blob.core.windows.net",
#   "ej8Nd4FyayPD9InmjCWZvVfB7nV4EWnkWu5IjWEUfs4aXRe7nz/xkPg6g1BtY6X5i+kHBQ8uDLGOG2PTpB28TQ==")
# df = spark.read.format("csv").option("inferSchema", "true").load("wasbs://hackathon@dlxhackathon1.blob.core.windows.net/")
# df.describe()

In [3]:
#spark.conf.set("spark.executor.cores",2)

df = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
  .options(**sfOptions) \
  .option("query", """select * from library_collection_inventory where reportdate in ('2017-09-01T00:00:00','2017-10-01T00:00:00', '2017-11-01T00:00:00', '2017-12-01T00:00:00', '2018-01-01T00:00:00', '2018-01-01T00:00:00', '2018-02-01T00:00:00', '2018-02-01T00:00:00', '2018-03-01T00:00:00', '2018-04-01T00:00:00', '2018-05-01T00:00:00', '2018-06-01T00:00:00', '2018-07-01T00:00:00') """) \
  .load().limit(200000)

NameError: name 'spark' is not defined

In [6]:
# Create a view or table

temp_table_name = "library_collection_inventory"

df.createOrReplaceTempView(temp_table_name)

In [7]:
# Import Libraries
import numpy as np
import pandas as pd
import nltk  # Has to be added through Workspaces/ attach library to cluster
import re
import os
import codecs
import mpld3
from sklearn import feature_extraction
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.pipeline import pipeline
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.feature_extraction.text import CountVectorizer



# pipeline = Pipeline(steps=[
#   ('vectorize', CountVectorizer()),
#   ('classify', DecisionTreeClassifier())
# ])

In [8]:
nltk.download('stopwords')  # Common words to ignore
nltk.download('punkt')  # Punkt Sentence Tokenizer

In [9]:
df_pandas = df.toPandas()  # Create pandas dataframe to work within python

# Drop the columns where any of the elements are missing values
df_pandas.dropna(axis=1, how='all')

# Drop the rows where any of the elements are missing values
df_pandas.dropna(axis=0, how='any')

# n = 30000  # Number of rows to analyse

titles = df_pandas.loc[0:n,'TITLE'].values.tolist()
subjects = df_pandas.loc[0:n,'SUBJECTS'].values.tolist()
author = df_pandas.loc[0:n, 'AUTHOR'].values.tolist()

# print(len(titles))

In [10]:
# df_pandas.loc[:,'PUBLICATIONYEAR'] = df_pandas.loc[:,'PUBLICATIONYEAR'].str.extract(r'(^|)*([0-9]{4})\s*(|$)', expand=True)

# _______________________________________________________________________________
# Functions for transforming the data
def ext_date_fun(string):
  string = string.str.extractstr.extract(r'(^|)*([0-9]{4})\s*(|$)', expand=True)
  
def fix_none_fun(sub):
  [str(item) for item in sub if item is None]
# _______________________________________________________________________________

# # Attempt at putting the functions into classes
# class PublicationYearCleaner(object):
#   """Preprocessing: This class cleans the Publication Year Column"""
#   def __init__(self, data):
#     self.raw = data
  
#   def ext_date(self, pub_year):
#      pub_year = pub_year.str.extract(r'(^|)*([0-9]{4})\s*(|$)', expand=True)

# class SubjectsCleaner(objct)
#   def fix_none(self, string):
#     if string is None:
#       return ''
#     return str(string)
# # ________________________________________________________________________________


In [11]:
# class ModelTransformer(TransformerMixin):
#     def __init__(self, model):
#         self.model = model

#     def fit(self, *args, **kwargs):
#         self.model.fit(*args, **kwargs)
#         return self

#     def transform(self, X, **transform_params):
#         return pd.DataFrame(self.model.predict(X))

In [12]:
# df_pandas.loc[:,'PUBLICATIONYEAR'] = df_pandas.loc[:,'PUBLICATIONYEAR'].str.extract(r'(^|)*([0-9]{4})\s*(|$)', expand=True)

ext_date_fun(df_pandas.loc[:,'PUBLICATIONYEAR'])
fix_none_fun(df_pandas.loc[:,'SUBJECTS'])

In [13]:
df_pandas = df_pandas.join(pd.get_dummies(df_pandas.loc[:,'ITEMTYPE']), how='inner')
df_pandas = df_pandas.join(pd.get_dummies(df_pandas.loc[:,'ITEMCOLLECTION']), how='inner')
df_pandas = df_pandas.join(pd.get_dummies(df_pandas.loc[:,'ITEMLOCATION']), how='inner')
list(df_pandas.columns.values)


Out[ 9 ]: 
['BIBNUM',
 'TITLE',
 'AUTHOR',
 'ISBN',
 'PUBLICATIONYEAR',
 'PUBLISHER',
 'SUBJECTS',
 'ITEMTYPE',
 'ITEMCOLLECTION',
 'FLOATINGITEM',
 'ITEMLOCATION',
 'REPORTDATE',
 'ITEMCOUNT',
 'acart',
 'acbk',
 'accas',
 'accd',
 'acdvd',
 'aceq',
 'ackit',
 'acmap',
 'acmus',
 'acpam',
 'acper',
 'acvhs',
 'ahbk',
 'alaptop',
 'arbk',
 'arcd',
 'arcdrom',
 'arkit',
 'armap',
 'armfc',
 'armus',
 'arpam',
 'arper',
 'arphoto',
 'arvhs',
 'atablet',
 'bcbk',
 'dcillb',
 'drord',
 'drper',
 'jcbk',
 'jccd',
 'jcdvd',
 'jckit',
 'jcmus',
 'jrbk',
 'jrcd',
 'pkbknh',
 'ucunkn',
 'caaero',
 'caaeroc',
 'caaeroo',
 'caaerop',
 'caatlas',
 'cab',
 'cabocd',
 'cabr',
 'cacd',
 'cacdnf',
 'cacdrr',
 'cacomic',
 'cacon',
 'cadesk3',
 'cadesk6',
 'cadesk7',
 'cadesk8',
 'cadesk9',
 'cadvd',
 'cadvdnf',
 'caeslr',
 'cafic',
 'cagr',
 'calibr',
 'caln',
 'calncas',
 'calncd',
 'calndvd',
 'calnr',
 'calpfic',
 'calpnf',
 'camapr',
 'camus',
 'camys',
 'canew',
 'canf',
 'caover',
 'capf',
 'careadr',
 'caref',
 'caref8',
 'caref9',
 'caref9f',
 'carefo',
 'casbr',
 'casear',
 'casf',
 'cash',
 'cass',
 'cawest',
 'ccaward',
 'ccb',
 'ccbocd',
 'cccd',
 'cccomic',
 'ccdesk',
 'ccdvd',
 'ccdvdnf',
 'ccef',
 'ccfft',
 'ccfic',
 'cchol',
 'cckit',
 'ccln',
 'cclpfic',
 'ccnf',
 'ccpic',
 'ccrdr',
 'ccref',
 'ccser',
 'cs1lewr',
 'cs1malf',
 'cs1malp',
 'cs3fic',
 'cs6r',
 'cs6rf',
 'cs6ro',
 'cs7r',
 'cs7rf',
 'cs7ro',
 'cs8r',
 'cs8ro',
 'cs8rx',
 'cs9g',
 'cs9gf',
 'cs9go',
 'cs9r',
 'cs9rf',
 'cs9ro',
 'cyb',
 'cybocd',
 'cycomic',
 'cyfic',
 'cylp',
 'cynf',
 'cyser',
 'naaab',
 'naaafic',
 'naaanew',
 'naaanf',
 'naaapb',
 'naaar',
 'naatlas',
 'naatlr',
 'nab',
 'nabocd',
 'nabr',
 'nacd',
 'nacdnf',
 'nacomic',
 'nacon',
 'naconr',
 'nadesk',
 'nadocr',
 'nadvd',
 'nadvdnf',
 'nafic',
 'naglc',
 'nahmwkr',
 'nahol',
 'najob',
 'najobr',
 'naln',
 'nalndvd',
 'nalpfic',
 'nalpnew',
 'nalpnf',
 'namar',
 'namys',
 'nanar',
 'nanew',
 'nanf',
 'naover',
 'napar',
 'napclbr',
 'napro',
 'naref',
 'narefsl',
 'nasf',
 'nass',
 'natxtr',
 'navidg',
 'nawest',
 'ncaafam',
 'ncaward',
 'ncb',
 'ncbocd',
 'nccd',
 'nccomic',
 'ncdvd',
 'ncdvdnf',
 'ncef',
 'ncenf',
 'ncfft',
 'ncfic',
 'nchol',
 'ncholsk',
 'ncln',
 'nclp',
 'ncnew',
 'ncnf',
 'ncpb',
 'ncpic',
 'ncrdr',
 'ncref',
 'ncser',
 'nyb',
 'nybocd',
 'nycomic',
 'nyfic',
 'nylp',
 'nynew',
 'nynf',
 'nypb',
 'nyref',
 'nyser',
 'pkfic',
 'pknf',
 'GWD',
 'bal',
 'bea',
 'bro',
 'cap',
 'cen',
 'col',
 'dlr',
 'dth',
 'fre',
 'glk',
 'gwd',
 'hip',
 'idc',
 'lcy',
 'mag',
 'mgm',
 'mob',
 'mon',
 'net',
 'nga',
 'nhy',
 'qna',
 'rbe',
 'spa',
 'swt',
 'tcs',
 'uni',
 'wal',
 'wts']

In [14]:
[str(i) for i in subjects]
#subjects_c = ["" for line in subjects if line.strip()]

Out[ 10 ]: 
['Labor unions, Employers associations, Industrial relations, Labor laws and legislation, Collective bargaining',
 'Colors Juvenile literature, Color, Childrens poetry',
 'Night Juvenile fiction, Birds Fiction, Animals Fiction, Winds Juvenile fiction',
 'Haristeen Harry Fictitious character Fiction, Murphy Mrs Fictitious character Fiction, Women detectives Virginia Fiction, Women cat owners Fiction, Cats Fiction, Large type books, Crozet Va Fiction, Mystery fiction',
 'Stepbrothers Drama, High school students Drama, Parent and child Drama, Television series, Childrens television programs, Video recordings for the hearing impaired',
 'Sports Fiction, Ability Juvenile fiction, Sports stories, Alligators Juvenile fiction, Humorous stories',
 'Monsters Fiction, Rabbits Fiction, Rain and rainfall Fiction, Thunderstorms Fiction',
 'Colorado River Colo Mexico Description and travel Juvenile literature, Colorado River Colo Mexico History Juvenile literature, Colorado River Colo Mexico Juvenile literature',
 'Mowgli Fictitious character Juvenile drama, Jungles Juvenile drama, Jungle animals Juvenile drama, Orphans Juvenile drama, Action and adventure films, Feature films, Fiction films, Childrens films, Animal films, Video recordings for the hearing impaired, Video recordings for people with visual disabilities',
 'Jews Italy Veneto History, Jews Italy Venice History, Jews Travel Italy Venice Guidebooks, Jews Travel Italy Veneto Guidebooks, Veneto Italy Guidebooks, Venice Italy Guidebooks',
 'Witches Juvenile drama, Riddles Juvenile drama, Balls Parties Juvenile drama, Feature films, Childrens films, Fantasy films, Fiction films, Video recordings for the hearing impaired',
 'Sisters Juvenile fiction, Babysitters Juvenile fiction, Behavior Fiction',
 'Groundhog Day Juvenile literature, Woodchuck Juvenile literature, Holidays Juvenile literature',
 'Teenage girls Juvenile fiction, Paranormal fiction, Ghost stories, Ghost stories, Young adult fiction',
 'Teenagers Fiction, Love stories, Bildungsromans, Humorous fiction',
 'Ranchers Fiction, Large type books, Western stories',
 'Seattle Wash History Chronology, King County Wash History Chronology, Washington Authors Collection',
 'Rock music 2011 2020',
 'Vikings Juvenile fiction, Dragons Juvenile fiction, Mice Juvenile fiction, Contests Juvenile fiction, Adventure stories, Adventure and adventurers Fiction',
 'Chinese fiction 21st century, Short stories Chinese',
 'United States Central Intelligence Agency History 20th century, Vietnam War 1961 1975 Campaigns Laos, Vietnam War 1961 1975 Secret service United States',
 'Veterans Drama, Assassins Drama, Criminals Drama, Television series, Thrillers Television programs, Fiction television programs, Video recordings for the hearing impaired',
 'Middle schools Fiction, Schools Fiction, Humorous stories, Voyages and travels Juvenile fiction, Friendship Juvenile fiction, Australia Juvenile fiction',
 'Man woman relationships Drama, Married people Drama, Sex role Drama, Padua Italy Drama, Comedies',
 'Christian rock music, Contemporary Christian music',
 'Elizabeth I Queen of England 1533 1603, Philip II King of Spain 1527 1598, Queens Great Britain Biography, Great Britain History Elizabeth 1558 1603, Great Britain Foreign relations Spain, Spain Foreign relations Great Britain',
 'American wit and humor',
 'Big Nate Comic books strips etc, Friendship Comic books strips etc, Middle schools Comic books strips etc, Schools Comic books strips etc, Humorous stories, Cartoons and comics, Comics Graphic works, Humorous comics',
 'Journalists Fiction, Serial murderers Fiction, Suspense fiction, Mystery fiction',
 'Bedtime Juvenile fiction, Sleep Juvenile fiction',
 'Tariff Law and legislation United States',
 'Popular instrumental music 2011 2020, Jazz 2011 2020, Saxophone music Jazz',
 'Double bass Methods, Double bass Studies and exercises',
 'Italian drama Translations into English',
 'Grey Joel 1932, Actors United States Biography, Photo

In [15]:
def xstr(s):
    if s is None:
        return ''
    return str(s)

In [4]:
subjects_c = []
for i in subjects:
  subjects_c.append(xstr(i))

df['subjects_c'] = 

NameError: name 'df_pandas' is not defined

In [17]:
for i in subjects_c:
  letters_only = re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          str(i))

In [18]:
print(titles[:10])

["Methods of collaboration between the public authorities, workers' organisations, and employers' organisations.", "What's your favorite color? / Eric Carle and friends.", 'The longest night / by Marion Dane Bauer ; illustrated by Ted Lewin.', 'Cat of the century / Rita Mae Brown & Sneaky Pie Brown ; illustrations by Michael Gellatly.', "Drake & Josh. Volume 1, Suddenly brothers / Schneider's Bakery ; Nickelodeon Productions ; produced by Keiren Fisher, Robin Weiner ; written by Dan Schneider ; directed by Virgil L. Fabian.", 'Aaron is a good sport / by P.D. Eastman.', 'Puddles!!! / Kevan Atteberry.', 'The Colorado River / Carol B. Rawlins.', 'The jungle book / Disney ; a Fairview Entertainment production ; produced by Jon Favreau, Brigham Taylor ; screenplay by Justin Marks ; directed by Jon Favreau.', 'Venice and environs : Jewish itineraries, places, history and art / edited by Francesa Brandes ; [translated from the Italian by Gus Barker].']

In [19]:
print(subjects_c[:10])

['Labor unions, Employers associations, Industrial relations, Labor laws and legislation, Collective bargaining', 'Colors Juvenile literature, Color, Childrens poetry', 'Night Juvenile fiction, Birds Fiction, Animals Fiction, Winds Juvenile fiction', 'Haristeen Harry Fictitious character Fiction, Murphy Mrs Fictitious character Fiction, Women detectives Virginia Fiction, Women cat owners Fiction, Cats Fiction, Large type books, Crozet Va Fiction, Mystery fiction', 'Stepbrothers Drama, High school students Drama, Parent and child Drama, Television series, Childrens television programs, Video recordings for the hearing impaired', 'Sports Fiction, Ability Juvenile fiction, Sports stories, Alligators Juvenile fiction, Humorous stories', 'Monsters Fiction, Rabbits Fiction, Rain and rainfall Fiction, Thunderstorms Fiction', 'Colorado River Colo Mexico Description and travel Juvenile literature, Colorado River Colo Mexico History Juvenile literature, Colorado River Colo Mexico Juvenile literature', 'Mowgli Fictitious character Juvenile drama, Jungles Juvenile drama, Jungle animals Juvenile drama, Orphans Juvenile drama, Action and adventure films, Feature films, Fiction films, Childrens films, Animal films, Video recordings for the hearing impaired, Video recordings for people with visual disabilities', 'Jews Italy Veneto History, Jews Italy Venice History, Jews Travel Italy Venice Guidebooks, Jews Travel Italy Veneto Guidebooks, Veneto Italy Guidebooks, Venice Italy Guidebooks']

In [20]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')
print(stopwords[:10])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [21]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [22]:
# Define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [23]:
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in subjects_c:
    allwords_stemmed = tokenize_and_stem(str(i)) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(str(i))
    totalvocab_tokenized.extend(allwords_tokenized)

In [24]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 400029 items in vocab_frame

In [25]:
print(vocab_frame.head())

words
labor labor
union unions
employ employers
associ associations
industri industrial

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=5, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(subjects_c) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 17.8 s, sys: 132 ms, total: 18 s
Wall time: 17.9 s
(30001, 19362)

In [27]:
terms = tfidf_vectorizer.get_feature_names()

for i in terms:
  print(i)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1435730652665579> in <module> () 
 ----> 1 terms = tfidf_vectorizer . get_feature_names ( ) 
 2 
 3 for i in terms : 
 4 print ( i ) 

 NameError : name 'tfidf_vectorizer' is not defined

In [28]:
from sklearn.metrics.pairwise import euclidean_distances
dist = 1 - euclidean_distances(tfidf_matrix)

In [29]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters, random_state=3425)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

print(clusters)

CPU times: user 58.9 s, sys: 32.8 ms, total: 58.9 s
Wall time: 59.2 s
[0, 3, 1, 1, 0, 1, 1, 3, 0, 0, 0, 1, 3, 1, 1, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 4, 1, 1, 2, 0, 0, 0, 2, 4, 2, 1, 0, 0, 4, 0, 1, 1, 4, 3, 1, 0, 3, 3, 0, 2, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 2, 1, 0, 0, 1, 1, 0, 1, 1, 1, 2, 0, 0, 2, 0, 3, 1, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 4, 0, 0, 2, 1, 4, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 2, 0, 1, 2, 2, 2, 0, 0, 1, 0, 1, 1, 3, 1, 0, 0, 0, 3, 2, 0, 0, 4, 0, 0, 3, 0, 0, 1, 0, 0, 0, 4, 2, 0, 2, 1, 0, 1, 3, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 4, 0, 3, 2, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 4, 1, 3, 1, 0, 0, 0, 0, 0, 0, 2, 1, 0, 1, 0, 1, 3, 1, 1, 0, 1, 0, 1, 2, 0, 3, 0, 0, 1, 0, 1, 1, 0, 0, 3, 2, 0, 0, 3, 3, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 3, 1, 0, 0, 3, 0, 1, 2, 3, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 2, 1, 1, 4, 2, 0, 0, 0, 0, 2, 0, 0, 1, 3, 1, 1, 1, 0, 0, 0, 0, 0, 1, 2, 0, 3, 0, 3, 0, 0, 0, 2, 0, 3, 0, 0, 0, 1, 2, 1, 2, 0, 0, 1, 3, 0, 0, 0, 0, 1, 1, 2, 0, 0, 4, 0, 0, 0, 1, 1, 0, 0, 0, 4, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 3, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 3, 4, 1, 1, 1, 0, 0, 0, 0, 1, 0, 3, 0, 1, 2, 0, 3, 3, 0, 1, 0, 0, 2, 0, 0, 3, 1, 2, 2, 0, 2, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 3, 1, 0, 0, 1, 2, 0, 4, 1, 3, 0, 0, 0, 2, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 3, 0, 1, 0, 2, 2, 0, 1, 0, 1, 4, 1, 2, 1, 1, 2, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 2, 0, 0, 0, 4, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 1, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 1, 1, 4, 1, 1, 3, 2, 1, 4, 0, 0, 0, 0, 1, 0, 4, 4, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 4, 0, 0, 0, 1, 0, 0, 1, 1, 4, 0, 3, 4, 2, 1, 0, 2, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 4, 1, 2, 0, 1, 2, 2, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 4, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 3, 0, 0, 2, 3, 2, 0, 1, 2, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 3, 0, 0, 1, 3, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 3, 1, 3, 0, 1, 1, 2, 1, 0, 1, 0, 1, 2, 0, 0, 0, 0, 3, 1, 0, 0, 3, 0, 3, 3, 0, 0, 4, 3, 0, 4, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 2, 2, 0, 0, 1, 1, 0, 1, 1, 3, 0, 0, 0, 1, 3, 0, 2, 0, 2, 0, 1, 2, 1, 0, 3, 0, 0, 3, 0, 0, 1, 0, 2, 0, 0, 2, 3, 0, 0, 1, 0, 4, 0, 1, 1, 4, 2, 0, 2, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 2, 3, 4, 0, 0, 4, 1, 3, 0, 1, 1, 0, 1, 0, 2, 0, 2, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 3, 0, 3, 1, 3, 1, 1, 0, 0, 1, 2, 1, 1, 1, 0, 0, 0, 0, 3, 1, 0, 1, 3, 2, 3, 0, 2, 0, 0, 0, 2, 0, 3, 1, 3, 0, 0, 2, 0, 0, 1, 1, 0, 1, 0, 4, 1, 4, 1, 0, 2, 3, 0, 1, 0, 1, 0, 0, 0, 1, 2, 1, 3, 0, 3, 1, 2, 0, 1, 0, 0, 0, 1, 2, 1, 0, 3, 1, 0, 1, 0, 3, 0, 1, 0, 0, 1, 0, 1, 0, 0, 3, 1, 0, 3, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 3, 1, 0, 1, 0, 2, 0, 0, 0, 0, 2, 1, 2, 0, 3, 1, 1, 2, 3, 0, 1, 3, 0, 0, 1, 1, 0, 1, 3, 0, 2, 1, 1, 0, 1, 2, 0, 1, 1, 0, 3, 0, 0, 1, 1, 0, 0, 1, 1, 1, 3, 0, 1, 0, 0, 0, 0, 4, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 3, 0, 0, 0, 0, 0, 0, 1, 0, 4, 0, 0, 0, 3, 4, 0, 2, 1, 1, 2, 1, 1, 0, 0, 4, 0, 0, 0, 0, 4, 0, 2, 3, 1, 0, 0, 0, 0, 0, 1, 2, 1, 3, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 2, 0, 0, 0, 2, 1, 1, 0, 1, 3, 0, 3, 4, 0, 1, 0, 0, 0, 0, 4, 3, 1, 0, 1, 0, 0, 3, 3, 1, 0, 1, 1, 0, 3, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 2, 1, 0, 0, 3, 1, 1, 0, 0, 0, 0, 0, 3, 0, 0, 0, 2, 1, 1, 2, 0, 0, 1, 0, 0, 2, 0, 0, 1, 0, 1, 0, 3, 0, 1, 0, 0, 0, 0, 1, 3, 4, 2, 1, 0, 2, 0, 0, 1, 1, 1, 1, 2, 0, 0, 3, 1, 1, 0, 3, 0, 4, 0, 0, 4, 2, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 3, 1, 1, 3, 0, 0, 0, 0, 0, 2, 3, 0, 0, 1, 0, 1, 2, 3, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 1, 0, 0, 3, 0, 2, 1, 0, 1, 0, 1, 0, 2, 3, 0, 1, 0, 2, 0, 0, 2, 3, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 2, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,

In [30]:
from sklearn.externals import joblib

#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle

joblib.dump(km,  'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [31]:
books = { 'title': titles, 'author': author, 'subjects': subjects_c, 'cluster': clusters }

frame = pd.DataFrame(books, index = [clusters] , columns = ['title', 'author', 'cluster'])

In [32]:
# frame.columns
frame.cluster.value_counts() #number of books per cluster (clusters from 0 to 4)

Out[ 28 ]: 
0 15168
1 8779
3 2506
2 2410
4 1138
Name: cluster, dtype: int64

In [33]:
from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
  print("Cluster %d words:" % i, end='')
    
  for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
    print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
#    print("Cluster %d titles:" % i, end='')
#    for title in frame.loc[i,'title'].values.tolist():
#        print(' %s,' % title, end='')

Top terms per cluster:

Cluster 0 words: b'music',

 b'drama',

 b'films',

 b'rock',

 b'rock',

 b'popular',

Cluster 1 words: b'fiction',

 b'juvenile',

 b'juvenile',

 b'stories',

 b'mystery',

 b'mystery',

Cluster 2 words: b'united',

 b'united',

 b'states',

 b'biography',

 b'states',

 b'united',

Cluster 3 words: b'juvenile',

 b'literature',

 b'juvenile',

 b'biography',

 b'biography',

 b'biography',

Cluster 4 words: b'comic',

 b'comic',

 b'books',

 b'strips',

 b'comic',

 b'books',

In [34]:
# import os  # for os.path.basename

# import matplotlib.pyplot as plt
# import matplotlib as mpl

# from sklearn.manifold import MDS

# MDS()

# # convert two components as we're plotting points in a two-dimensional plane
# # "precomputed" because we provide a distance matrix
# # we will also specify `random_state` so the plot is reproducible.
# mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)

# pos = mds.fit_transform(dist)  # shape (n_components, n_samples)

# xs, ys = pos[:, 0], pos[:, 1]

In [35]:
dummies = pd.get_dummies(frame.cluster)
dummies.columns = dummies.columns.astype(str)
list(dummies.columns.values)


Out[ 31 ]: ['0', '1', '2', '3', '4']

In [36]:
# dummies.rename(index=str, columns={"0": "Juvenile Literature", "1": "Music (Country)", "2": "Mystery", "3": "Music (Rock)", "4": "Comic Books"})
dummies.columns = ["Drama / Film / Rock", "Juvinile Mystery", "United States / Biography", "Juvinile Literature / Biography", "Comic Books"]
list(dummies.columns.values)

Out[ 36 ]: 
['Drama / Film / Rock',
 'Juvinile Mystery',
 'United States / Biography',
 'Juvinile Literature / Biography',
 'Comic Books']

In [37]:
#frame
df_pandas_dummies = df_pandas.join(dummies, how='left')
df_pandas_dummies
list(df_pandas_dummies.columns.values)

Out[ 37 ]: 
['BIBNUM',
 'TITLE',
 'AUTHOR',
 'ISBN',
 'PUBLICATIONYEAR',
 'PUBLISHER',
 'SUBJECTS',
 'ITEMTYPE',
 'ITEMCOLLECTION',
 'FLOATINGITEM',
 'ITEMLOCATION',
 'REPORTDATE',
 'ITEMCOUNT',
 'acart',
 'acbk',
 'accas',
 'accd',
 'acdvd',
 'aceq',
 'ackit',
 'acmap',
 'acmus',
 'acpam',
 'acper',
 'acvhs',
 'ahbk',
 'alaptop',
 'arbk',
 'arcd',
 'arcdrom',
 'arkit',
 'armap',
 'armfc',
 'armus',
 'arpam',
 'arper',
 'arphoto',
 'arvhs',
 'atablet',
 'bcbk',
 'dcillb',
 'drord',
 'drper',
 'jcbk',
 'jccd',
 'jcdvd',
 'jckit',
 'jcmus',
 'jrbk',
 'jrcd',
 'pkbknh',
 'ucunkn',
 'caaero',
 'caaeroc',
 'caaeroo',
 'caaerop',
 'caatlas',
 'cab',
 'cabocd',
 'cabr',
 'cacd',
 'cacdnf',
 'cacdrr',
 'cacomic',
 'cacon',
 'cadesk3',
 'cadesk6',
 'cadesk7',
 'cadesk8',
 'cadesk9',
 'cadvd',
 'cadvdnf',
 'caeslr',
 'cafic',
 'cagr',
 'calibr',
 'caln',
 'calncas',
 'calncd',
 'calndvd',
 'calnr',
 'calpfic',
 'calpnf',
 'camapr',
 'camus',
 'camys',
 'canew',
 'canf',
 'caover',
 'capf',
 'careadr',
 'caref',
 'caref8',
 'caref9',
 'caref9f',
 'carefo',
 'casbr',
 'casear',
 'casf',
 'cash',
 'cass',
 'cawest',
 'ccaward',
 'ccb',
 'ccbocd',
 'cccd',
 'cccomic',
 'ccdesk',
 'ccdvd',
 'ccdvdnf',
 'ccef',
 'ccfft',
 'ccfic',
 'cchol',
 'cckit',
 'ccln',
 'cclpfic',
 'ccnf',
 'ccpic',
 'ccrdr',
 'ccref',
 'ccser',
 'cs1lewr',
 'cs1malf',
 'cs1malp',
 'cs3fic',
 'cs6r',
 'cs6rf',
 'cs6ro',
 'cs7r',
 'cs7rf',
 'cs7ro',
 'cs8r',
 'cs8ro',
 'cs8rx',
 'cs9g',
 'cs9gf',
 'cs9go',
 'cs9r',
 'cs9rf',
 'cs9ro',
 'cyb',
 'cybocd',
 'cycomic',
 'cyfic',
 'cylp',
 'cynf',
 'cyser',
 'naaab',
 'naaafic',
 'naaanew',
 'naaanf',
 'naaapb',
 'naaar',
 'naatlas',
 'naatlr',
 'nab',
 'nabocd',
 'nabr',
 'nacd',
 'nacdnf',
 'nacomic',
 'nacon',
 'naconr',
 'nadesk',
 'nadocr',
 'nadvd',
 'nadvdnf',
 'nafic',
 'naglc',
 'nahmwkr',
 'nahol',
 'najob',
 'najobr',
 'naln',
 'nalndvd',
 'nalpfic',
 'nalpnew',
 'nalpnf',
 'namar',
 'namys',
 'nanar',
 'nanew',
 'nanf',
 'naover',
 'napar',
 'napclbr',
 'napro',
 'naref',
 'narefsl',
 'nasf',
 'nass',
 'natxtr',
 'navidg',
 'nawest',
 'ncaafam',
 'ncaward',
 'ncb',
 'ncbocd',
 'nccd',
 'nccomic',
 'ncdvd',
 'ncdvdnf',
 'ncef',
 'ncenf',
 'ncfft',
 'ncfic',
 'nchol',
 'ncholsk',
 'ncln',
 'nclp',
 'ncnew',
 'ncnf',
 'ncpb',
 'ncpic',
 'ncrdr',
 'ncref',
 'ncser',
 'nyb',
 'nybocd',
 'nycomic',
 'nyfic',
 'nylp',
 'nynew',
 'nynf',
 'nypb',
 'nyref',
 'nyser',
 'pkfic',
 'pknf',
 'GWD',
 'bal',
 'bea',
 'bro',
 'cap',
 'cen',
 'col',
 'dlr',
 'dth',
 'fre',
 'glk',
 'gwd',
 'hip',
 'idc',
 'lcy',
 'mag',
 'mgm',
 'mob',
 'mon',
 'net',
 'nga',
 'nhy',
 'qna',
 'rbe',
 'spa',
 'swt',
 'tcs',
 'uni',
 'wal',
 'wts',
 'Drama / Film / Rock',
 'Juvinile Mystery',
 'United States / Biography',
 'Juvinile Literature / Biography',
 'Comic Books']

In [38]:
prediction = df_pandas.loc[(n+1):,:]

In [39]:
X_train, X_test, y_train, y_test = train_test_split(df_pandas_dummies, dummies, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-1435730652665593> in <module> () 
 ----> 1 X_train , X_test , y_train , y_test = train_test_split ( df_pandas_dummies , dummies , test_size = 0.2 ) 
 2 print ( X_train . shape , y_train . shape ) 
 3 print ( X_test . shape , y_test . shape ) 

 /databricks/python/lib/python3.5/site-packages/sklearn/model_selection/_split.py in train_test_split (*arrays, **options) 
 1687 test_size = 0.25 
 1688 
 -> 1689 arrays = indexable ( * arrays ) 
 1690 
 1691 if stratify is not None : 

 /databricks/python/lib/python3.5/site-packages/sklearn/utils/validation.py in indexable (*iterables) 
 204 else : 
 205 result . append ( np . array ( X ) ) 
 --> 206 check_consistent_length ( * result ) 
 207 return result 
 208 

 /databricks/python/lib/python3.5/site-packages/sklearn/utils/validation.py in check_consistent_length (*arrays) 
 179 if len ( uniques ) > 1 : 
 180 raise ValueError("Found input variables with inconsistent numbers of"
 --> 181 " samples: %r" % [int(l) for l in lengths])
 182 
 183 

 ValueError : Found input variables with inconsistent numbers of samples: [229996, 30001]